In [1]:
import numpy as np

In [2]:
# from https://zpl.fi/aligning-point-patterns-with-kabsch-umeyama-algorithm/
def kabsch_umeyama(src, dst):
    assert dst.shape == src.shape
    n, m = dst.shape

    dst_mean = np.mean(dst, axis=0)
    src_mean = np.mean(src, axis=0)
    VarA = np.mean(np.linalg.norm(dst - dst_mean, axis=1) ** 2)

    H = ((dst - dst_mean).T @ (src - src_mean)) / n
    U, D, VT = np.linalg.svd(H)
    d = np.sign(np.linalg.det(U) * np.linalg.det(VT))
    S = np.diag([1] * (m - 1) + [d])

    R = U @ S @ VT
    c = VarA / np.trace(np.diag(D) @ S)
    t = dst_mean - c * R @ src_mean

    T = np.eye(m + 1, dtype=np.float64)
    T[:m, :m] = c * R
    T[:m, m] = t
    return T

In [3]:
# From https://github.com/scikit-image/scikit-image/blob/54c2148455d8b30463724706e6a5fe09b4f52172/skimage/transform/_geometric.py#L91
def _umeyama(src, dst, estimate_scale):
    """Estimate N-D similarity transformation with or without scaling.
    Parameters
    ----------
    src : (M, N) array_like
        Source coordinates.
    dst : (M, N) array_like
        Destination coordinates.
    estimate_scale : bool
        Whether to estimate scaling factor.
    Returns
    -------
    T : (N + 1, N + 1)
        The homogeneous similarity transformation matrix. The matrix contains
        NaN values only if the problem is not well-conditioned.
    References
    ----------
    .. [1] "Least-squares estimation of transformation parameters between two
            point patterns", Shinji Umeyama, PAMI 1991, :DOI:`10.1109/34.88573`
    """
    src = np.asarray(src)
    dst = np.asarray(dst)

    num = src.shape[0]
    dim = src.shape[1]

    # Compute mean of src and dst.
    src_mean = src.mean(axis=0)
    dst_mean = dst.mean(axis=0)

    # Subtract mean from src and dst.
    src_demean = src - src_mean
    dst_demean = dst - dst_mean

    # Eq. (38).
    A = dst_demean.T @ src_demean / num

    # Eq. (39).
    d = np.ones((dim,), dtype=np.float64)
    if np.linalg.det(A) < 0:
        d[dim - 1] = -1

    T = np.eye(dim + 1, dtype=np.float64)

    U, S, V = np.linalg.svd(A)

    # Eq. (40) and (43).
    rank = np.linalg.matrix_rank(A)
    if rank == 0:
        return np.nan * T
    elif rank == dim - 1:
        if np.linalg.det(U) * np.linalg.det(V) > 0:
            T[:dim, :dim] = U @ V
        else:
            s = d[dim - 1]
            d[dim - 1] = -1
            T[:dim, :dim] = U @ np.diag(d) @ V
            d[dim - 1] = s
    else:
        T[:dim, :dim] = U @ np.diag(d) @ V

    if estimate_scale:
        # Eq. (41) and (42).
        scale = 1.0 / src_demean.var(axis=0).sum() * (S @ d)
    else:
        scale = 1.0

    T[:dim, dim] = dst_mean - scale * (T[:dim, :dim] @ src_mean.T)
    T[:dim, :dim] *= scale

    return T

In [4]:
A = np.array([[ 23, 178],
              [ 66, 173],
              [ 88, 187],
              [119, 202],
              [122, 229],
              [170, 232],
              [179, 199]])
B = np.array([[232, 38],
              [208, 32],
              [181, 31],
              [155, 45],
              [142, 33],
              [121, 59],
              [139, 69]])

In [5]:
T1 = _umeyama(A, B, True) 
T2 = kabsch_umeyama(A, B) 

In [6]:
T1, T2

(array([[ -0.55439848,  -0.40088362, 309.2086726 ],
        [  0.40088362,  -0.55439848, 110.81144783],
        [  0.        ,   0.        ,   1.        ]]),
 array([[ -0.60130945,  -0.43480478, 321.13300577],
        [  0.43480478,  -0.60130945, 116.47685146],
        [  0.        ,   0.        ,   1.        ]]))

In [7]:
def apply_T(T, data):
    x = (T[0, 0] * data[:, 0]) + (T[0, 1] * data[:, 1]) + T[0, 2]
    y = (T[1, 1] * data[:, 1]) + (T[1, 0] * data[:, 0]) + T[1, 2]
    return np.array([x, y]).T 

In [8]:
apply_T(T1, A) - B

array([[ -6.89977746, -16.65115853],
       [ -4.73449404,   9.35882969],
       [  4.45636863,  11.41669067],
       [  7.25676136,   1.52810577],
       [  7.76970808,  -0.23800236],
       [  0.9559301 ,  -8.65878387],
       [ -8.80449666,   3.24431863]])

In [9]:
apply_T(T2, A) - B

array([[ -2.09236259, -18.55572034],
       [ -1.77464495,   9.14743249],
       [  5.90928025,  11.2948054 ],
       [  6.74661564,   1.7541119 ],
       [  6.2029582 ,  -1.17682886],
       [ -2.96430965,  -8.11012771],
       [-12.0275369 ,   5.64632711]])